In [1]:
# Gỡ cài đặt Keras hiện tại
!pip uninstall -y keras

# Cài đặt phiên bản Keras cụ thể
!pip install keras==2.*

# Cài đặt các phụ thuộc còn thiếu
!pip install wurlitzer

Found existing installation: keras 3.4.1
Uninstalling keras-3.4.1:
  Successfully uninstalled keras-3.4.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [2]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires typeguard<5,>=4.1.2, but you have typeguard 2.13.3 which is incompatible.


# Lib

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

2024-07-31 15:43:40.125438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 15:43:40.125539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 15:43:40.254773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

# Dữ liệu

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [5]:
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [6]:
num_classes = 10
input_shape = (32, 32, 3)

# Tăng cường dữ liệu ảnh

In [7]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# thiết lập trạng thái lớp chuẩn hóa
data_augmentation.layers[0].adapt(x_train)

# Mô hình

In [8]:
def create_encoder():
    res_net = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = res_net(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model

In [9]:
encoder = create_encoder()
encoder.summary()

Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23564807 (89.89 MB)
Trainable params: 23519360 (89.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________


In [10]:
learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

# Mô hình phân loại

In [11]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [12]:
encoder = create_encoder()
classifier = create_classifier(encoder)
classifier.summary()

Model: "cifar10-classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functiona  (None, 2048)              23564807  
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                

In [13]:
history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/50


I0000 00:00:1722440665.907843      90 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


189/189 [==============================] - 66s 136ms/step - loss: 1.8468 - sparse_categorical_accuracy: 0.3303
Epoch 2/50
189/189 [==============================] - 18s 95ms/step - loss: 1.4134 - sparse_categorical_accuracy: 0.4921
Epoch 3/50
189/189 [==============================] - 18s 97ms/step - loss: 1.2450 - sparse_categorical_accuracy: 0.5625
Epoch 4/50
189/189 [==============================] - 19s 99ms/step - loss: 1.1014 - sparse_categorical_accuracy: 0.6174
Epoch 5/50
189/189 [==============================] - 19s 101ms/step - loss: 1.0019 - sparse_categorical_accuracy: 0.6556
Epoch 6/50
189/189 [==============================] - 19s 100ms/step - loss: 0.9429 - sparse_categorical_accuracy: 0.6742
Epoch 7/50
189/189 [==============================] - 19s 99ms/step - loss: 0.8471 - sparse_categorical_accuracy: 0.7114
Epoch 8/50
189/189 [==============================] - 19s 99ms/step - loss: 0.8265 - sparse_categorical_accuracy: 0.7186
Epoch 9/50
189/189 [====================

# Sử dụng Học tương phản có giám sát

In [14]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Chuẩn hóa các vector đặc trưng
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Tính logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)

In [15]:
def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model

# Huấn luyện

In [16]:
encoder = create_encoder()
encoder_projection_head = add_projection_head(encoder)
encoder_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_projection_head.summary()

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functiona  (None, 2048)              23564807  
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
Total params: 23827079 (90.89 MB)
Trainable params: 23781632 (90.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________


In [17]:
history = encoder_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Epoch 1/50
189/189 [==============================] - 43s 99ms/step - loss: 5.3294
Epoch 2/50
189/189 [==============================] - 19s 101ms/step - loss: 5.0799
Epoch 3/50
189/189 [==============================] - 19s 100ms/step - loss: 4.9304
Epoch 4/50
189/189 [==============================] - 19s 98ms/step - loss: 4.8189
Epoch 5/50
189/189 [==============================] - 19s 98ms/step - loss: 4.7084
Epoch 6/50
189/189 [==============================] - 19s 99ms/step - loss: 4.6089
Epoch 7/50
189/189 [==============================] - 19s 99ms/step - loss: 4.5337
Epoch 8/50
189/189 [==============================] - 19s 99ms/step - loss: 4.4563
Epoch 9/50
189/189 [==============================] - 19s 99ms/step - loss: 4.3853
Epoch 10/50
189/189 [==============================] - 19s 99ms/step - loss: 4.3285
Epoch 11/50
189/189 [==============================] - 19s 99ms/step - loss: 4.2769
Epoch 12/50
189/189 [==============================] - 19s 99ms/step - loss: 4.2191

# Huấn luyện phân loại với frozen encoder

In [18]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/50
189/189 [==============================] - 8s 25ms/step - loss: 0.1554 - sparse_categorical_accuracy: 0.9627
Epoch 2/50
189/189 [==============================] - 5s 25ms/step - loss: 0.1189 - sparse_categorical_accuracy: 0.9689
Epoch 3/50
189/189 [==============================] - 5s 25ms/step - loss: 0.1110 - sparse_categorical_accuracy: 0.9695
Epoch 4/50
189/189 [==============================] - 5s 25ms/step - loss: 0.1079 - sparse_categorical_accuracy: 0.9708
Epoch 5/50
189/189 [==============================] - 5s 25ms/step - loss: 0.1062 - sparse_categorical_accuracy: 0.9701
Epoch 6/50
189/189 [==============================] - 5s 26ms/step - loss: 0.1070 - sparse_categorical_accuracy: 0.9703
Epoch 7/50
189/189 [==============================] - 5s 26ms/step - loss: 0.1077 - sparse_categorical_accuracy: 0.9694
Epoch 8/50
189/189 [==============================] - 5s 25ms/step - loss: 0.1040 - sparse_categorical_accuracy: 0.9697
Epoch 9/50
189/189 [====================